In [1]:
import os,io
from google.cloud import vision
import numpy as np
import pandas as pd
import math
#from PIL import Image

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'ServiceAccountToken.json'
client = vision.ImageAnnotatorClient()

bounds = []
#img = Image.open(os.path.join(r'D:\SiH-Frame-Project\MyVisionDemo\Images','dining_room_1.jpg'))
with io.open(os.path.join(r'D:\SiH-Frame-Project\MyVisionDemo\Images','object_test1.jpeg'), 'rb') as image_file:
    content = image_file.read()

image = vision.types.Image(content=content)

In [3]:
#img.size

In [4]:
response = client.object_localization(image=image)

In [5]:
response.localized_object_annotations

[mid: "/m/01c648"
name: "Laptop"
score: 0.6757840514183044
bounding_poly {
  normalized_vertices {
    x: 0.8397461771965027
    y: 0.5542725324630737
  }
  normalized_vertices {
    x: 0.9963159561157227
    y: 0.5542725324630737
  }
  normalized_vertices {
    x: 0.9963159561157227
    y: 0.8072665333747864
  }
  normalized_vertices {
    x: 0.8397461771965027
    y: 0.8072665333747864
  }
}
, mid: "/m/01mzpv"
name: "Chair"
score: 0.6376141309738159
bounding_poly {
  normalized_vertices {
    x: 0.44442594051361084
    y: 0.6487604975700378
  }
  normalized_vertices {
    x: 0.7102159857749939
    y: 0.6487604975700378
  }
  normalized_vertices {
    x: 0.7102159857749939
    y: 0.9956747889518738
  }
  normalized_vertices {
    x: 0.44442594051361084
    y: 0.9956747889518738
  }
}
, mid: "/m/04bcr3"
name: "Table"
score: 0.6203668713569641
bounding_poly {
  normalized_vertices {
    x: 0.44676992297172546
    y: 0.5396133065223694
  }
  normalized_vertices {
    x: 0.990584135055542

In [6]:
frame = pd.DataFrame(columns=['object','confidence','x1','y1','x2','y2','x3','y3','x4','y4','centreX','centreY','xLoc','yLoc'])

In [7]:
def getXLoc(centreX):
    return 'Left' if centreX<0.33 else 'Centre' if centreX<0.66 else 'Right'
def getYLoc(centreY):
    return 'Top' if centreY<0.33 else 'Centre' if centreY<0.66 else 'Bottom'

In [8]:
obj_detail = list()
for loc_obj in response.localized_object_annotations:
    obj_detail.clear()
    for vertex in loc_obj.bounding_poly.normalized_vertices:
        obj_detail.append(vertex.x)
        obj_detail.append(vertex.y)
    frame_dict = dict(object = loc_obj.name,
                             confidence = loc_obj.score,
                             x1 = obj_detail[0],
                             y1 = obj_detail[1],
                             x2 = obj_detail[2],
                             y2 = obj_detail[3],
                             x3 = obj_detail[4],
                             y3 = obj_detail[5],
                             x4 = obj_detail[6],
                             y4 = obj_detail[7],
                             centreX = (obj_detail[0]+obj_detail[4])/2,
                             centreY = (obj_detail[1]+obj_detail[5])/2
                             )
    loc_dict = dict(xLoc = getXLoc(frame_dict.get('centreX')) , yLoc = getYLoc(frame_dict.get('centreY')))
    frame_dict.update(loc_dict)
    frame = frame.append(frame_dict,ignore_index=True)

In [9]:
frame

,object,confidence,x1,y1,x2,y2,x3,y3,x4,y4,centreX,centreY,xLoc,yLoc
0,Laptop,0.675784,0.839746,0.554273,0.996316,0.554273,0.996316,0.807267,0.839746,0.807267,0.918031,0.680770,Right,Bottom
1,Chair,0.637614,0.444426,0.648760,0.710216,0.648760,0.710216,0.995675,0.444426,0.995675,0.577321,0.822218,Centre,Bottom
2,Table,0.620367,0.446770,0.539613,0.990584,0.539613,0.990584,0.991216,0.446770,0.991216,0.718677,0.765415,Right,Bottom
3,Furniture,0.551563,0.108042,0.079800,0.449345,0.079800,0.449345,0.572901,0.108042,0.572901,0.278693,0.326350,Left,Top
